In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com/quiz-tatabahasa.jsonl

In [2]:
from tqdm import tqdm
import os
import json
import random

In [3]:
apikey = ''

In [4]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {apikey}',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
        'model': 'gpt-4-1106-preview',
    }

    response = requests.post(
        'https://api.openai.com/v1/chat/completions',
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [5]:
questions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        data = {
            'no': no,
            'objektif': 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction'],
            'soalan': '\n'.join(soalan),
            'jawapan': jawapan,
        }
        questions.append(data)
len(questions)

349

In [6]:
arange = set(range(len(questions)))

In [7]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """
    return prompt.strip()

In [8]:
i = 0
shots = random.sample(arange - {i}, 3)
prompts = []
for no, s in enumerate(shots):
    prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts)
print(prompt)

Contoh soalan 1
objektif: Pilih perkataan yang paling sesuai untuk menggantikan perkataan bergaris.
soalan: Kadir mengajak rakan-rakannya menonton siaran langsung perlawanan bola sepak di rumahnya.
A. mempelawa
B. meminta
C. melambai
D. memaksa
jawapan: A

Contoh soalan 2
objektif: Jawab soalan yang diberikan
soalan: Burung helang bolehlah dikatakan ....................... dalam golongan haiwan pemangsa.
A. memasuki
B. termasuk
C. dimasuki
D. dimasukkan
jawapan: B

Contoh soalan 3
objektif: Jawab soalan yang diberikan
soalan: Menurut Ketua Polis Negara, ada beberapa cara untuk ............ jenayah.
A. membenteras
B. melenyapkan
C. membasmi
D. menghapuskan
jawapan: A

objektif: Jawab soalan yang diberikan
soalan: ........, sudah dapat memandu kereta rupa-rupanya kamu !
A. Oh
B. Eh
C. Hai
D. Ah


In [9]:
r = curl(prompt)

In [10]:
r.json()['choices'][0]['message']['content']

'jawapan: B'

In [11]:
for i in tqdm(range(len(questions))):
    shots = random.sample(arange - {i}, 3)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|█████████████████████████████████████████| 349/349 [04:37<00:00,  1.26it/s]


In [12]:
import json

with open('output-3shot.json', 'w') as fopen:
    json.dump(questions, fopen)

In [13]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

349

In [14]:
correct = 0
for q in filtered:
    correct += q['output'].replace('jawapan:', '').split('.')[0].strip() == q['jawapan']
(correct / len(filtered)) * 100

75.64469914040114